# Dominando o SQL para análise de dados 👩‍💻

***

## Motivação

Quais as linguagens mais utilizadas por profissionais de dados atualmente ?

![State of Data 2021](../img/aula00/dataset_2021.png)
![State of Data 2022](../img/aula00/dataset_2022.png)

> Datasets completos disponíveis na pasta `datasets`.

```python
import pandas as pd
import numpy as np
import io
import re

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')

def read_file_and_get_info(_file_path: str, _file_name: str) -> pd.DataFrame:

    # Read .csv file
    df = pd.read_csv(_file_path)

    # Save DataFrame info to .txt file
    str_buffer = io.StringIO()
    df.info(verbose=True, buf=str_buffer)
    s = str_buffer.getvalue()

    with open(f'../datasets/{_file_name}', 'w', encoding='utf-8') as f:  
        f.write(s)

    return df

def get_P4_data(df: pd.DataFrame, source_col: str) -> pd.DataFrame:
    
    # Select data from P4, question 'd' - (Quais das linguagens listadas abaixo você utiliza no trabalho?)
    df_P4_cols = [c for c in df.columns if re.search('P4_d_', c)]
    df_P4 = df.loc[:, df_P4_cols].copy()
    df_P4.columns = [eval(c)[1] for c in df_P4_cols]
    df_P4 = df_P4.dropna(how='all', axis=0)
    df_P4['Source'] = source_col
    
    return df_P4

# Read and extract data from files
df_2021 = read_file_and_get_info('../datasets/State of Data 2021 - Dataset - Pgina1.csv', 'df_2021_info.txt')
df_2022 = read_file_and_get_info('../datasets/State_of_data_2022.csv', 'df_2022_info.txt')

df_2021_P4 = get_P4_data(df_2021, '2021')
df_2022_P4 = get_P4_data(df_2022, '2022')
df = pd.concat([df_2021_P4, df_2022_P4])

# Create plot
f, ax = plt.subplots(figsize=(8, 12))

data = df.groupby('Source').sum().melt(ignore_index=False, var_name='Linguagem', value_name='Total').sort_values(by='Total', ascending=False)
s = sns.barplot(data=data, x='Total', y='Linguagem', hue='Source', orient='h', palette='husl')

for container in ax.containers:
    s.bar_label(container, fontsize=10, padding=5, alpha=.75, bbox=dict(facecolor='0.95', boxstyle='round'))

ax.legend(loc='lower right', title='')
ax.set(ylabel='', xlabel='')
plt.title('Linguagens utilizadas (State of Data Brasil)', fontsize=16)
plt.tight_layout()
plt.savefig('../img/barplot.png')
```

```python
totals = data.groupby(level=0)['Total'].sum()

data_stacked = data.copy()
data_stacked['Linguagem'] = data_stacked['Linguagem'].apply(lambda x: 'SQL + Python' if x in ['SQL', 'Python'] else 'Outro')
data_stacked = data_stacked.reset_index().groupby(['Source', 'Linguagem'])['Total'].sum().reset_index()
data_stacked = data_stacked.merge(right=totals, left_on='Source', right_index=True, suffixes=('', '_agg'))
data_stacked['Perc'] = data_stacked['Total'] / data_stacked['Total_agg'] * 100

# Create plot
ax = sns.histplot(data_stacked, x='Source', hue='Linguagem', weights='Perc', multiple='stack', palette='Accent', shrink=0.8)
ax.set(ylabel='Percentual', xlabel='', title='Linguagens mais utilizadas (State of Data Brasil)')

legend = ax.get_legend()
legend.set_title('')
legend.set_bbox_to_anchor((1, 1))

plt.tight_layout()
plt.savefig('../img/stackedplot.png')
```

![image](../img/aula00/barplot.png)
![image](../img/aula00/stackedplot.png)

## O que é SQL ? 🗃

SQL significa _structured-query-language_ e é uma linguagem de programação para armazenar e processar informações em um **banco de dados relacional**.

👍 Muito bom para:

- Armazenamento eficaz de dados (especialmente em grande escala);
- Otimizado para operações de leitura/escrita;
- Manuseio eficaz de dados num formato adequado para análise;
    - Realização de cálculos simples
- Segurança (restrições de acesso e integridade dos dados)

👎 Não é tão bom para:

- Personalizar a formatação dos resultados das tabelas (Excel/Python/R são melhores para tal);
- Criação de gráficos (Excel/Python/R novamente);
- Cálculos complexos
- Elaboração de relatórios (Word/Python/R ...);
- Mapas (Python/R/QGIS);
    - Tecnicamente, pode-se plotar mapas em SQL, mas extremamente básicos

_Quando se diz "dados em grande escala", significa dizer "dados que excedem a quantidade de memória/RAM do computador em que você está fazendo sua análise"._

> **Dica:** - SQL não é _case sensitive_

![SQL meme woman at cat image](https://live.staticflickr.com/65535/49136979022_bcbf5443aa_z.jpg)

## SQL *vs.* NoSQL

Os bancos de dados NoSQL podem ser chamados de "não relacionais" (*não SQL*) são desenhados para processar grandes volumes de dados não estruturados ou cuja estrutura está em constante mudança.

Essencialmente a proposta é de armazenar informação em estruturas diferentes de um banco de dados relacional (SQL) com linhas e tabelas.

| Vantagens      | Desvantagens    |
|----------------|-----------------|
| Flexibilidade  | Restrições ACID |
| Escalabilidade | Complexidade    |
| Performance    |                 |

Exemplos de bancos de dados NoSQL:

![NoSQL Types](../img/aula00/nosql.png)

Recomendações de leitura: 📖
- [Transações ACID](https://www.databricks.com/br/glossary/acid-transactions)
- [NoSQL Storage](https://medium.com/@projectdomainbusiness/nosql-storage-4315fe0afab9)
- [O que é NoSQL?](https://aws.amazon.com/pt/nosql/)

## Como usar SQL? 💃

Considerando que este treinamento terá como base um conjunto de **Notebooks**, é recomendável que o **Azure Data Studio** seja utilizado. No entanto outras ferramentas como o **SQL Server Management Studio (SSMS)** e **VS Code** podem ser utilizadas.

### Instalando o SQL Server

1. [Instalar o SQL Server Express 2019](https://www.microsoft.com/pt-br/download/details.aspx?id=101064)
2. Configuração da instância local:
   1. [Guia de Instalação](../manuais/sqlserver.md)

### Configurar ferramentas

1. [Instalar o SQL Server Management Studio (SSMS)](https://learn.microsoft.com/pt-br/sql/ssms/download-sql-server-management-studio-ssms?view=sql-server-ver16)
2. [Instalar o Azure Data Studio](https://go.microsoft.com/fwlink/?linkid=2247827)

### Restaurar banco de dados de treinamento SQL_TREINAMENTO
   1. [Guia de Restauração](../manuais/backup_banco_de_dados.md)

### Conectar ao banco de dados local

Após a instalação e configuração da instância de SQL Server e da restauração do *backup* do banco de dados de treinamento, é possível se conectar aos dados:
- Servidor: **<localhost\INSTANCIA>**
- Base de dados: **SQL_TREINAMENTO**

O banco de dados *SQL_TREINAMENTO* é uma versão descaracterizada de um banco de dados real de um projeto HGA e será nosso *playground*.

***

## Próximas aulas 👁

1. Aula 01: Fundamentos de Modelagem de dados e SQL
   - Modelagemde dados básica
   - Introdução à Linguagem SQL
   - Manipulação básica de dados
   - Desafio 01
2. Aula 02: Manipulação de dados com SQL
   - Resolução do Desafio 01
   - Funções de agregação
   - Mesclagem de tabelas (JOIN e UNION)
   - Tabelas temporárias
   - Desafio 02
3. Aula 03: Técnicas avançadas de consultas
   - Resolução do Desafio 02
   - Ranking e Window funcions
   - Pivot e Unpivot
   - Exemplos reais
   - Desafio 03
4. Aula 04: Melhores práticas de SQL
   - Dicas de SQL (Sintaxe, funcionalidades gerais)
   - Performance
   - Mais exemplos práticos
   - Desafio final
***